In [6]:
import sqlite3
import plotly.graph_objs as go
from plotly.offline import plot 
from pprint import pprint

def selectToList(query):
    db = sqlite3.connect('../data/location-comparative.sqlite')
    sql = db.cursor()
    sql.execute(query)
    results = list()
    for r in sql.fetchall():
        results.append(r[0])
    #pprint(results)
    return results 

def listToBox(values,n):
    #TODO set colour based on Firefox/Tor Original/Ublock?
    return go.Box(y=values,name=n,boxpoints=False)#,jitter=0.3,pointpos=-1.8)


def drawBoxes(boxes,title):
    fig = go.Figure()
    for b in boxes:
        fig.add_trace(b)

    fig.update_layout(showlegend=False)

    fig.update_layout(
        title=go.layout.Title(
            text=title,
            xref="paper",
            x=0,
            font=dict(
                size=24,
            )
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text="Configuration",
            font=dict(
                size=24,
            )
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text="Time (Milliseconds)",
            font=dict(
                size=24,
            )
            )
        )
    )
    fig.update_yaxes(range=[0,30000])
    fig.update_xaxes(tickangle=45, tickfont=dict(size=18))
    fig.update_yaxes(tickangle=0, tickfont=dict(size=18))
    fig.show()

In [8]:
locations = selectToList("SELECT DISTINCT location from results")
locations = [l.split('--')[0]+'--'+l.split('--')[1] for l in locations] #Drop the exact instance number, use the zone+browser

boxes = list()
for l in locations:
    if 'tor' in l:
        boxes.append(listToBox(selectToList("""
        SELECT visualComplete85 FROM results WHERE step == 0 AND location LIKE '"""+l+"""%' AND label LIKE '%ublock%';
    """), l+'and Ublock, First Load'))
drawBoxes(boxes,'Tor and Ublock')

In [9]:
boxes = list()
for l in locations:
    if 'tor' not in l:
        boxes.append(listToBox(selectToList("""
        SELECT visualComplete85 FROM results WHERE step == 0 AND location LIKE '"""+l+"""%' AND label LIKE '%ublock%';
    """), l+'and Ublock, First Load'))
drawBoxes(boxes,'Firefox and Ublock')

In [10]:
boxes = list()
for l in locations:
    if 'tor' in l:
        boxes.append(listToBox(selectToList("""
        SELECT visualComplete85 FROM results WHERE step == 0 AND location LIKE '"""+l+"""%' AND label LIKE '%original%';
    """), l+'and Original, First Load'))
drawBoxes(boxes,'Tor and Original')

In [11]:
boxes = list()
for l in locations:
    if 'tor' not in l:
        boxes.append(listToBox(selectToList("""
        SELECT visualComplete85 FROM results WHERE step == 0 AND location LIKE '"""+l+"""%' AND label LIKE '%original%';
    """), l+'and Original, First Load'))
drawBoxes(boxes,'Firefox and Original')
